1. Importing Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import pickle
from scipy import sparse,spatial,stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#### Getting the Dataset
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2021-09-19 05:56:32--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  10.2MB/s    in 0.5s    

2021-09-19 05:56:32 (10.2 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [ ]:
#### Creating the Train Dataset
trainData = pd.read_csv("ml-100k/u1.base",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
trainData.head(5)

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [ ]:
########## Number of unique users & unique movies
print("Number of unique Users: "+str(len(trainData['user_id'].unique())))
print("Number of unique Movies: "+str(len(trainData['item_id'].unique())))

Number of unique Users: 943
Number of unique Movies: 1650


In [ ]:
####### Utility Matrix
utilityMatrix = csr_matrix((943, 1682), dtype=np.float)
for index, row in trainData.iterrows():
    # print(f"{index+1} row and rating {float(row['rating'])}")
    utilityMatrix[row['user_id']-1,row['item_id']-1]=float(row['rating'])


In [ ]:
sparse.save_npz("utilityMatrix.npz",utilityMatrix)

In [ ]:
utilityMatrix=sparse.load_npz("utilityMatrix.npz")

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

1.Item-Item Similarity

1.1 Jaccard Similarity

In [ ]:
#For jaccard similarity we have to generate User sets like movie1={user1,user2,user5} for all movies and Movie sets like user1={movie1,movie2,movie10}
movieSets = dict.fromkeys(range(1,1683))
for key in movieSets.keys():
  movieSets[key]=set()

for index, row in trainData.iterrows():
    if row['user_id'] not in movieSets[row['item_id']]:
      movieSets[row['item_id']].add(row['user_id'])


userSets = dict.fromkeys(range(1,944))
for key in userSets.keys():
  userSets[key]=set()

for index, row in trainData.iterrows():
    if row['item_id'] not in userSets[row['user_id']]:
      userSets[row['user_id']].add(row['item_id'])

In [ ]:
# Calculate Jaccard Similarity between all movies and save that as a sparse matrix
itemItemJaccard = csr_matrix((1682, 1682), dtype=np.float)

In [ ]:
def calculateJaccard(movie1,movie2):
  intersection=len(movieSets[movie1] & movieSets[movie2])
  union=float(len(movieSets[movie1] | movieSets[movie2]))
  if (movie1==movie2):
    return 0
  if (union==0):
    return 0
  else:
    return intersection/union

for movie1 in range(1,1683):
  #### Calculate jaccard between movie1 and movie2:
  similarityScoreList = [calculateJaccard(movie1,movie2) for movie2 in range(1,1683)]
  itemItemJaccard[movie1-1]=list(similarityScoreList)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
########## N most similar movies rated by same user
def MostSimilarMovies(M,movie,user,n):
  usersMovieList=userSets[user]
  similarMovieList=[i+1 for i in list(np.argsort(M[movie].toarray())[0][::-1])]
  nSimilarMoviesRatedByUser=[]
  counter=0
  for similarMovie in similarMovieList:
    if similarMovie in usersMovieList:
      nSimilarMoviesRatedByUser.append(similarMovie)
      counter+=1
    if counter==n:
      break
  return nSimilarMoviesRatedByUser

In [ ]:
########## Rating Function
def calculateRating(M,movie,user,n):
  listSimilarMovies=MostSimilarMovies(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  for similarmovie in listSimilarMovies:
    weightedSimilaritySum+=M[movie-1,similarmovie-1]*utilityMatrix[user-1,similarmovie-1]
    similaritySum+=M[movie-1,similarmovie-1]
  rating=weightedSimilaritySum/similaritySum
  return rating

In [ ]:
def is_relevant(rating):
  if rating>3.5:
    return 1
  else:
    return 0

def average_precision_k(dataset,k=10,threshold=3):
  # dataset['relevant_actual']=dataset['rating'].map(is_relevant)
  # dataset['relevant_predicted']=dataset['predicted_rating'].map(is_relevant)
  precisions=[]
  for user in range(943):
    dataframeUser = dataset[dataset['user_id']==user+1]
    dataframeUserActual = dataframeUser.sort_values(by='rating',ascending=False)
    dataframeUserPredicted = dataframeUser.sort_values(by='predicted_rating',ascending=False)
    actualReleventSet=set()
    predictedReleventSet=set()
    actualCounter=0
    predictedCounter=0
    for index,row in dataframeUserActual.iterrows():
      if(actualCounter==k):
        break
      if(row['rating']>=threshold):
        actualCounter+=1
        actualReleventSet.add(row['item_id'])
    for index,row in dataframeUserPredicted.iterrows():
      if(predictedCounter==k):
        break
      if(row['predicted_rating']>=threshold):
        predictedCounter+=1
        predictedReleventSet.add(row['item_id'])   
    if(len(actualReleventSet)==0):
      userPrecisionK==1
    else:
      userPrecisionK = len(actualReleventSet & predictedReleventSet)/len(actualReleventSet)   
    precisions.append(userPrecisionK)
  return sum(precisions)/len(precisions)

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

1.1.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(itemItemJaccard,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.283691844403821

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7358283256745608

1.1.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(itemItemJaccard,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.1409504379757387

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7421493376424447

1.1.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(itemItemJaccard,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.079171360219829

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7501317140500597

1.2 Cosine Similarity

In [ ]:
movieSets = dict.fromkeys(range(1,1683))
for key in movieSets.keys():
  movieSets[key]=set()

for index, row in trainData.iterrows():
    if row['user_id'] not in movieSets[row['item_id']]:
      movieSets[row['item_id']].add(row['user_id'])


userSets = dict.fromkeys(range(1,944))
for key in userSets.keys():
  userSets[key]=set()

for index, row in trainData.iterrows():
    if row['item_id'] not in userSets[row['user_id']]:
      userSets[row['user_id']].add(row['item_id'])

In [ ]:
CosineSimilarity = cosine_similarity(utilityMatrix.transpose(),dense_output=False)

1.2.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(CosineSimilarity,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2756917559743732

=>Precision@10

In [ ]:
average_precision_k(testData,k=50,threshold=3)

0.22692353010486624

1.2.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(CosineSimilarity,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.1463560641199184

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7436179703411943

1.2.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(CosineSimilarity,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


=>Squared Error

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0945642667833613

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7505159150970392

1.3 Pearson correlation coefficient

In [ ]:
PearsonCorrelation = pd.DataFrame(utilityMatrix.toarray()).corr(method='pearson').values
PearsonCorrelation = sparse.csr_matrix(PearsonCorrelation)

1.3.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

12.546825995028968

=>Precision@10

In [ ]:
average_precision_k(testData,k=50,threshold=3)

0.35901331166771044

1.3.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=>Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

4.454566102200838

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.23114889999831661

1.3.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRating(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=>Squared Error

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

20.68206732667912

=>Precision@10

In [ ]:
average_precision_k(testData,k=10,threshold=3)

0.7502625864767963

In [ ]:
def np_pearson_cor(x, y):
    xv = x - x.mean(axis=0)
    yv = y - y.mean(axis=0)
    xvss = (xv * xv).sum(axis=0)
    yvss = (yv * yv).sum(axis=0)
    result = np.matmul(xv.transpose(), yv) / np.sqrt(np.outer(xvss, yvss))
    # bound the values to -1 to 1 in the event of precision issues
    return np.maximum(np.minimum(result, 1.0), -1.0)

In [ ]:
PearsonCorrelation = pd.DataFrame(utilityMatrix.toarray()).corr(method='pearson').values
PearsonCorrelation = sparse.csr_matrix(PearsonCorrelation)


In [ ]:
header=pd.MultiIndex.from_product([['Item-Item'],
                                     ['Jaccard Similarity','Cosine Similarity','Pearson Correlation'],['RMSE','Precision@10'],['N=2','N=5','N=10']],
                                    names=['Algorithm','Similarity Function','Metric','Neighbours'])
resultsTable=pd.DataFrame(np.reshape(np.array([1.28,1.14,1.07,0.73,0.74,0.75,1.27,1.14,1.09,0.22,0.74,0.75,12.54,4.45,20.68,0.35,0.23,0.75]),(1,18)),columns=header)

In [ ]:
display(resultsTable)

Algorithm                    Item-Item              ...                                
Similarity Function Jaccard Similarity              ... Pearson Correlation            
Metric                            RMSE              ...        Precision@10            
Neighbours                         N=2   N=5  N=10  ...                 N=2   N=5  N=10
0                                 1.28  1.14  1.07  ...                0.35  0.23  0.75

[1 rows x 18 columns]

In [ ]:
1.28,1.14,1.07
0.73,0.74,0.75

1.27,1.14,1.09
0.22,0.74,0.75

12.54,4.45,20.68
0.35,0.23,0.75

2.User User Similarity

2.1 Jaccard Similarty

In [ ]:

userUserJaccard = csr_matrix((942, 942), dtype=np.float)

In [ ]:
def calculateJaccardUserUser(user1,user2):
  intersection=len(userSets[user1] & userSets[user2])
  union=float(len(userSets[user1] | userSets[user2]))
  if (user1==user2):
    return 0
  if (union==0):
    return 0
  else:
    return intersection/union

for user1 in range(1,943):
  #### Calculate jaccard between user1 and user2:
  similarityScoreList = [calculateJaccard(user1,user2) for user2 in range(1,943)]
  userUserJaccard[user1-1]=list(similarityScoreList)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
calculateJaccardUserUser.data[np.isnan(calculateJaccardUserUser.data)] = 0.5

In [ ]:
userUserJaccard.shape

(942, 942)

In [ ]:
########## N most similar users who rated the same movie
def MostSimilarUser(M,movie,user,n):
  movieUserList=movieSets[movie]
  similarUserList=[i+1 for i in list(np.argsort(M[user].toarray())[0][::-1])]
  nSimilarUsersRatedMovie=[]
  counter=0
  for similarUser in similarUserList:
    if similarUser in movieUserList:
      nSimilarUsersRatedMovie.append(similarUser)
      counter+=1
    if counter==n:
      break
  return nSimilarUsersRatedMovie

In [ ]:
########## Rating Function
def calculateRatingUserUser(M,movie,user,n):
  # listSimilarUsers=MostSimilarUser(M,movie,user,n)
  # similarUsersRatings=0
  # for similarUser in listSimilarUsers:
  #   similarUsersRatings+=utilityMatrix[similarUser-1,movie-1]
  # averageRating=similarUsersRatings/float(n)
  # return averageRating
  listSimilarUsers=MostSimilarUser(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  for similarUser in listSimilarUsers:
    weightedSimilaritySum+=M[user-1,similarUser-1]*utilityMatrix[similarUser-1,movie-1]
    similaritySum+=M[user-1,similarUser-1]
  if similaritySum==0:
    return 0
  rating=weightedSimilaritySum/similaritySum
  return rating

2.1.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(userUserJaccard,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=> Mean Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2757607552645365

=>Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.23536080391859804

2.1.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(userUserJaccard,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=> Mean Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.134489614633833

=>Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2399421804777055

2.1.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(userUserJaccard,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

=> Mean Squared Error

In [ ]:

mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0816563629240898

=>Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.25067498190509846

**2.2 Cosine Similarity**

In [ ]:
CosineSimilarity = cosine_similarity(utilityMatrix,dense_output=False)

2.2.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(CosineSimilarity,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
## n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2638535178880494

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.23558341328754914

2.2.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(CosineSimilarity,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.1342784639157588

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.7526502718443331

2.2.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(CosineSimilarity,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.0841042327720172

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.2497180561867731

**2.3 Pearson Correlation**

In [ ]:
PearsonCorrelation = pd.DataFrame(utilityMatrix.transpose().toarray()).corr(method='pearson').values
PearsonCorrelation = sparse.csr_matrix(PearsonCorrelation)

2.3.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
## n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

2.065238576055945

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.2323141274891008

2.3.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
## n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.720197037003922

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.7544955309801544

2.3.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

===> Root mean Squared

In [ ]:
## n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2498001960098646

===> Precision @10

In [ ]:
average_precision_k(testData,k=10)

0.2500639633725531

In [ ]:
header=pd.MultiIndex.from_product([['User-User'],
                                     ['Jaccard Similarity','Cosine Similarity','Pearson Correlation'],['RMSE','Precision@10'],['N=2','N=5','N=10']],
                                    names=['Algorithm','Similarity Function','Metric','Neighbours'])
resultsTable=pd.DataFrame(np.reshape(np.array([1.27,1.13,1.08,0.23,0.23,0.25,1.26,1.13,1.08,0.23,0.75,0.24,2.06,1.72,1.24,0.23,0.75,0.25]),(1,18)),columns=header)

In [ ]:
display(resultsTable)

Algorithm                    User-User              ...                                
Similarity Function Jaccard Similarity              ... Pearson Correlation            
Metric                            RMSE              ...        Precision@10            
Neighbours                         N=2   N=5  N=10  ...                 N=2   N=5  N=10
0                                 1.27  1.13  1.08  ...                0.23  0.75  0.25

[1 rows x 18 columns]